# Aula 08

# Trabalhando com NoSQL no Python

### Instalando o PyMongo:

### Acessando o AtlasDB

Após a criação da conta no Atlas e configuração do nosso banco de dados. Vamos estabelecer a conexão:

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


uri = "mongodb+srv://<username>:<password>@<clustername>/"

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


In [ ]:
# Seleciona o banco de dados
db = client["aula07"]

# Seleciona a coleção
collection = db["cotacoes"]

Vamos baixar os 10 últimos anos da cotação do dólar aqui no Brasil.



In [2]:
import yfinance as yf
import json

# Defina os pares das moedas no formato correto para forex
symbols = ["USDBRL=X"]

# Defina o período de busca (1 ano)
start_date = "2014-04-21"
end_date = "2024-04-24"

# Crie um dicionário vazio para armazenar os dados
data = {}

# Para cada símbolo, extraia os dados e adicione ao dicionário
for symbol in symbols:
    # Baixe os dados históricos
    historical_data = yf.download(symbol, start=start_date, end=end_date)

    # Verifique se os dados foram baixados com sucesso
    if not historical_data.empty:
        # Converta as chaves do DataFrame de Timestamp para string
        historical_data.index = historical_data.index.strftime('%Y-%m-%d')
        # Converta o DataFrame em um dicionário
        symbol_data = historical_data.to_dict()

        # Adicione os dados ao dicionário principal
        data[symbol] = symbol_data

# Salve o dicionário em um arquivo JSON
with open("cotacoes.json", "w") as outfile:
    json.dump(data, outfile, indent=4)



[*********************100%%**********************]  1 of 1 completed


In [ ]:
import json

# Carrega os dados do arquivo JSON
with open("cotacoes.json", "r") as file:
    data = json.load(file)

Adicionando o json ao banco de dados outra tabela do banco:

In [ ]:
collection.insert_one(data)

InsertOneResult(ObjectId('66270162f0905ab0546bcda0'), acknowledged=True)

### Usando os dados para fazer previsão de cenários futuros

Agora dispomos de um serviço de banco de dados na nuvem que contém um arquivo específico. Em seguida, simularemos o acesso a esse arquivo para realizar previsões de cenários futuros.

O dataset em questão representa a cotação do dólar (em reais) ao longo dos últimos 10 anos.

Primeiro, vamos iniciar o processo acessando o JSON que estava no banco de dados.

Dentro do arquivo JSON, vamos acessar a tag que contém a cotação do dólar de fechamento da bolsa.

Para visualizar o gráfico da cotação do dólar, podemos utilizar a biblioteca `matplotlib`, uma das ferramentas mais populares para a criação de gráficos em Python. Ela oferece uma ampla gama de funcionalidades que permitem a construção de gráficos estáticos, animados e interativos de maneira simples e eficaz. Através do `matplotlib`, é possível customizar gráficos com títulos, legendas e escalas, tornando-os uma escolha versátil para análise e apresentação de dados.

Para realizar previsões de cenários futuros, podemos utilizar a biblioteca `Prophet`, desenvolvida pelo Facebook. O Prophet é especializado em previsões para séries temporais, sendo robusto a dados faltantes e mudanças de tendência, e capaz de lidar com efeitos sazonais de diferentes granularidades. Essa ferramenta é particularmente útil para dados que exibem padrões sazonais fortes e várias temporadas de histórico. O Prophet facilita a modelagem de séries temporais, permitindo que analistas e desenvolvedores produzam previsões confiáveis com facilidade.

Exibir componentes da previsão (tendência, sazonalidade)

Esta é uma solução inicial, e percebemos que o modelo ainda não está captando a tendência de alta na cotação que está ocorrendo atualmente. Muitos ajustes são necessários para melhorar a qualidade do ajuste. Ajustar parâmetros, incorporar variáveis externas e refinar a modelagem sazonal são algumas das estratégias que podem ser exploradas para aprimorar as previsões do modelo.

Por exemplo, podemos informar ao modelo as datas que foram feriados no Brasil e os períodos que tivemos crise financeira (levei em consideração o período da copa/olimpíadas 2014-2016 e o período da covid 2021-2022).

Com os ajustes realizados, houve uma perda de precisão nos dados históricos, porém, os dados mais recentes passaram a se alinhar com a faixa esperada. É importante destacar que este modelo ainda está distante de ser considerado confiável. Para alcançar um modelo mais preciso, seria essencial realizar um estudo aprofundado em séries temporais e economia, o que permitiria refinar as técnicas de modelagem e integrar conhecimentos específicos do mercado financeiro.